# Importing necessary libraries

In [29]:
import numpy as np 
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, classification_report
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Reading Data

In [30]:
df = pd.read_csv(r"C:\Users\ajayk\Desktop\Machine Learning\Machine-Learning\Malware-Detection\ClaMP_Integrated-5184.csv")
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,sus_sections,non_sus_sections,packer,packer_type,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,256,4,1,0,1,...,1,3,0,NoPacker,6.603616,5.443362,1181520,6.627552,1,0
1,144,3,4,65535,184,184,4,1,0,1,...,1,3,0,NoPacker,5.205926,2.123522,7680,5.318221,0,0
2,144,3,4,65535,184,272,5,1,0,1,...,1,4,0,NoPacker,6.238000,3.380859,57872,6.507758,1,0
3,144,3,4,65535,184,184,1,1,0,1,...,0,1,0,NoPacker,0.000000,0.000000,95616,4.575092,1,0
4,144,3,4,65535,184,224,5,1,0,1,...,1,4,0,NoPacker,6.355626,0.702621,48128,5.545531,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,216,5,1,1,1,...,0,5,0,NoPacker,6.174602,3.155928,365568,7.546568,0,1
5206,80,2,4,65535,184,256,7,1,0,1,...,4,3,0,NoPacker,0.000000,0.000000,98816,6.947195,0,1
5207,144,3,4,65535,184,216,5,1,0,1,...,2,3,0,NoPacker,6.503422,3.790871,227328,7.823114,0,1
5208,144,3,4,65535,184,248,5,1,1,1,...,1,4,0,NoPacker,6.115208,7.919091,271616,7.886012,0,1


In [31]:
df_integrated_data = pd.read_csv((r"C:\Users\ajayk\Desktop\Machine Learning\Machine-Learning\Malware-Detection\ClaMP_Integrated-5184.csv"), skip_blank_lines=True, na_filter=False, encoding='utf-8')

print('Total Shape :',df_integrated_data.shape)

Total Shape : (5210, 70)


In [32]:
df_integrated_data.head()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,sus_sections,non_sus_sections,packer,packer_type,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,256,4,1,0,1,...,1,3,0,NoPacker,6.603616,5.443362,1181520,6.627552,1,0
1,144,3,4,65535,184,184,4,1,0,1,...,1,3,0,NoPacker,5.205926,2.123522,7680,5.318221,0,0
2,144,3,4,65535,184,272,5,1,0,1,...,1,4,0,NoPacker,6.238000,3.380859,57872,6.507758,1,0
3,144,3,4,65535,184,184,1,1,0,1,...,0,1,0,NoPacker,0.000000,0.000000,95616,4.575092,1,0
4,144,3,4,65535,184,224,5,1,0,1,...,1,4,0,NoPacker,6.355626,0.702621,48128,5.545531,1,0


# Checking columns and their data type

In [33]:
for col in df_integrated_data.columns:
  print('COLUMN:', col, 'DATA_TYPE:', df_integrated_data[col].dtype, 'UNIQUE_VAL:', df_integrated_data[col].nunique())

COLUMN: e_cblp DATA_TYPE: int64 UNIQUE_VAL: 10
COLUMN: e_cp DATA_TYPE: int64 UNIQUE_VAL: 8
COLUMN: e_cparhdr DATA_TYPE: int64 UNIQUE_VAL: 4
COLUMN: e_maxalloc DATA_TYPE: int64 UNIQUE_VAL: 4
COLUMN: e_sp DATA_TYPE: int64 UNIQUE_VAL: 10
COLUMN: e_lfanew DATA_TYPE: int64 UNIQUE_VAL: 41
COLUMN: NumberOfSections DATA_TYPE: int64 UNIQUE_VAL: 22
COLUMN: CreationYear DATA_TYPE: int64 UNIQUE_VAL: 2
COLUMN: FH_char0 DATA_TYPE: int64 UNIQUE_VAL: 2
COLUMN: FH_char1 DATA_TYPE: int64 UNIQUE_VAL: 1
COLUMN: FH_char2 DATA_TYPE: int64 UNIQUE_VAL: 2
COLUMN: FH_char3 DATA_TYPE: int64 UNIQUE_VAL: 2
COLUMN: FH_char4 DATA_TYPE: int64 UNIQUE_VAL: 2
COLUMN: FH_char5 DATA_TYPE: int64 UNIQUE_VAL: 2
COLUMN: FH_char6 DATA_TYPE: int64 UNIQUE_VAL: 2
COLUMN: FH_char7 DATA_TYPE: int64 UNIQUE_VAL: 2
COLUMN: FH_char8 DATA_TYPE: int64 UNIQUE_VAL: 2
COLUMN: FH_char9 DATA_TYPE: int64 UNIQUE_VAL: 2
COLUMN: FH_char10 DATA_TYPE: int64 UNIQUE_VAL: 2
COLUMN: FH_char11 DATA_TYPE: int64 UNIQUE_VAL: 1
COLUMN: FH_char12 DATA_TYPE: 

# Checking null values on each column in the dataset

In [34]:
df_integrated_data.isna().sum()/len(df_integrated_data)*100

e_cblp        0.0
e_cp          0.0
e_cparhdr     0.0
e_maxalloc    0.0
e_sp          0.0
             ... 
E_data        0.0
filesize      0.0
E_file        0.0
fileinfo      0.0
class         0.0
Length: 70, dtype: float64

# Changed the 'packer_type' column to 'pt_num' with the appropriate integer

In [35]:
number = df_integrated_data['packer_type'].unique()
p_types = {number[i] : i for i in range(len(number))}
temp = []
for n in df_integrated_data['packer_type']:
    temp.append(p_types[n])
df_integrated_data['pt_num'] = temp
y = df_integrated_data.pop('class')
df_integrated_data.pop('packer_type')

0       NoPacker
1       NoPacker
2       NoPacker
3       NoPacker
4       NoPacker
          ...   
5205    NoPacker
5206    NoPacker
5207    NoPacker
5208    NoPacker
5209    NoPacker
Name: packer_type, Length: 5210, dtype: object